## SVM

In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from path import Path
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# load data
file_path = Path('./Resources/ml_clean_stroke_dataset.csv')
stroke_df = pd.read_csv(file_path)
stroke_df.head()

,Age,Gender,Ever Married,Smoking Status,Hypertension,Heart Disease,Stroke
0,3,Male,No,unknown,0,0,0
1,58,Male,Yes,never smoked,1,0,0
2,8,Female,No,unknown,0,0,0
3,70,Female,Yes,formerly smoked,0,0,0
4,14,Male,No,unknown,0,0,0


## Preprocess data

In [4]:
# Use scikit-learn to encode data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

stroke_df['Gender'] = le.fit_transform(stroke_df['Gender'])
stroke_df['Ever Married'] = le.fit_transform(stroke_df['Ever Married'])
stroke_df['Smoking Status'] = le.fit_transform(stroke_df['Smoking Status'])
stroke_df.head()

,Age,Gender,Ever Married,Smoking Status,Hypertension,Heart Disease,Stroke
0,3,1,0,3,0,0,0
1,58,1,1,1,1,0,0
2,8,0,0,3,0,0,0
3,70,0,1,0,0,0,0
4,14,1,0,3,0,0,0


## Scaling & Normalization

In [5]:
# load scaler
data_scaler = StandardScaler()

In [6]:
# train the scaler and transform the data
stroke_data_scaled = data_scaler.fit_transform(stroke_df)

In [7]:
# preview scaled data
stroke_data_scaled[:5]

array([[-1.73913268,  1.20108119, -1.34420288,  1.28640708, -0.32129564,
        -0.22334159, -0.13554685],
       [ 0.70066001,  1.20108119,  0.74393532, -0.53968634,  3.11239826,
        -0.22334159, -0.13554685],
       [-1.51733334, -0.83084068, -1.34420288,  1.28640708, -0.32129564,
        -0.22334159, -0.13554685],
       [ 1.23297842, -0.83084068,  0.74393532, -1.45273305, -0.32129564,
        -0.22334159, -0.13554685],
       [-1.25117414,  1.20108119, -1.34420288,  1.28640708, -0.32129564,
        -0.22334159, -0.13554685]])

In [8]:
# verify the mean
print(np.mean(stroke_data_scaled[:,0]))
print(np.std(stroke_data_scaled[:,0]))

-1.0314330035227261e-16
0.9999999999999999


####  Standardization was  successful, and all numerical columns now have a mean of 0 and a standard deviation of 1, reducing the likelihood that large values will unduly influence the model.

## Split Data into Training and Testing

In [9]:
# Create our features
X = pd.get_dummies(stroke_df.drop(columns="Stroke"))

# Create our target
y= stroke_df["Stroke"]

In [10]:
# Check the balance of our target values
y.value_counts()

0    42617
1      783
Name: Stroke, dtype: int64

In [11]:
# Split into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
X_train.shape

(32550, 6)

In [13]:
X_test.shape

(10850, 6)

## Oversample Data

In [14]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 31963, 1: 31963})

## Run SVM

In [15]:
# import SVC module
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [16]:
# train the model
model.fit(X_train, y_train)

SVC(kernel='linear')

In [17]:
# create predictions
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,0
4,0,0


In [18]:
# assess the accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9819354838709677

In [19]:
# generate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     10654
           1       0.00      0.00      0.00       196

    accuracy                           0.98     10850
   macro avg       0.49      0.50      0.50     10850
weighted avg       0.96      0.98      0.97     10850



In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5

In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      1.00      0.00      0.99      0.00      0.00     10654
          1       0.00      0.00      1.00      0.00      0.00      0.00       196

avg / total       0.96      0.98      0.02      0.97      0.00      0.00     10850



In [22]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,10654,0
Actual 1,196,0
